In [10]:
import {deepSeekChatModel} from './models/index.ts'
import {zodToJsonSchema }from 'zod-to-json-schema'
import {z} from 'zod'
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableSequence } from "@langchain/core/runnables";
import {JsonOutputToolsParser} from '@langchain/core/output_parsers/openai_tools'
const classifySchema = z.object({
    type: z.enum(["科普", "编程", "一般问题"]).describe("用户提问的分类")
})
const model = deepSeekChatModel.bind({  temperature: 0 })
const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "classifyQuestion",
                description: "对用户的提问进行分类",
                parameters: zodToJsonSchema(classifySchema),
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "classifyQuestion"
        }
    }
})
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，
    当你无法分类到特定分类时，可以分类到 "一般问题"`],
    ["human", "{input}"]
])
const classifyChain = RunnableSequence.from([
    prompt,
    modelWithTools,
   new JsonOutputToolsParser(),
    (input) => {
        const type = input[0]?.args?.type
        return type ? type : "一般问题"
    }
])
await classifyChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})


"科普"

In [12]:
await classifyChain.invoke({
    "input": "编写一个python代码生成斐波那契数列"
})


"编程"

In [15]:
import { StringOutputParser } from "@langchain/core/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";
import { RunnableBranch  } from "@langchain/core/runnables";
const answeringModel = deepSeekChatModel.bind({  temperature: 0.7 })

const sciencePrompt = PromptTemplate.fromTemplate(
  `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：

问题：{input}
答案：`
)
    
const programmingPrompt = PromptTemplate.fromTemplate(
  `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：

问题：{input}
答案：`
)

const generalPrompt = PromptTemplate.fromTemplate(
  `请回答以下一般性问题，尽可能提供全面和有深度的答案：

问题：{input}
答案：`
)


const scienceChain = RunnableSequence.from([
    sciencePrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "科普专家"
    }
    
])

const programmingChain = RunnableSequence.from([
    programmingPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "编程大师"
    }
    
])

const generalChain = RunnableSequence.from([
    generalPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "通识专家"
    }
    
])
const branch = RunnableBranch.from([
  [
    (input => input.type.includes("科普")),
    scienceChain,
  ],
  [
    (input => input.type.includes("编程")),
    programmingChain,
  ],
  generalChain
]);

const outputTemplate = PromptTemplate.fromTemplate(
`感谢您的提问，这是来自 {role} 的专业回答：

{output}
`)


const finalChain = RunnableSequence.from([
    {
        type: classifyChain,
        input: input => input.input
    },
    branch,
    (input) => outputTemplate.format(input),
])
const res = await finalChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

res

"感谢您的提问，这是来自 科普专家 的专业回答：\n" +
  "\n" +
  "是的，鲸鱼是哺乳动物。尽管它们生活在水中，外形与鱼类相似，但鲸鱼属于哺乳动物纲（Mammalia）中的鲸目（Cetacea）。以下是鲸鱼作为哺乳动物的几个关键特征：\n" +
  "\n" +
  "1. **恒温性**：鲸鱼是恒温动物，能够维持相对稳定的体温，这与鱼类不同，鱼类是变温动物。\n" +
  "\n" +
  "2. **呼吸方式**：鲸鱼通过肺呼吸，而不是通过鳃。它们需要定期浮出水面呼吸空气，通过头顶的喷水孔（blowhole）进行呼吸。\n" +
  "\n" +
  "3. **胎生和哺乳**：鲸鱼是胎生动物，幼鲸在母体内发育成熟后出生。出生后，幼鲸会通过哺乳的方式获取营养，母鲸会分泌乳汁喂养幼鲸。\n" +
  "\n" +
  "4. **毛发**：虽然成年鲸鱼体表几乎没有毛发，但它们在胚胎期和幼年期会有少量毛发，这是哺乳动物的一个特征。\n" +
  "\n" +
  "5. **心脏结构**：鲸鱼的心脏有四腔室，与所有哺乳动物一样，这有助于高效地输送氧气和营养物质。\n" +
  "\n" +
  "6. **骨骼结构**：鲸鱼的骨骼结构与陆生哺乳动物相似，尤其是它们的鳍状肢内部结构与哺乳动物的四肢骨骼相似。\n" +
  "\n" +
  "7. **社会行为**：鲸鱼表现出复杂的社会行为和交流方式，这也是哺乳动物的一个特征。\n" +
  "\n" +
  "综上所述，鲸鱼虽然生活在水中，但它们具备哺乳动物的所有关键特征，因此被归类为哺乳动物。\n"